# DIY Covid-19 Dashboard

In [37]:
from IPython.display import clear_output #imports all libraries that were used for the dashboard
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [38]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [39]:
with open("england.json", "rt") as INFILE:
    jsondata = json.load(INFILE) #This opens the json file from the directory, assigns to variable




In [40]:
#This function parses date string into pandas datetime object
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

# This function wrangles the data and is used by both graphs 
def wrangle_data(rawdata):
    datalist = rawdata['data']
    dates = [entry['date'] for entry in datalist]
    dates.sort()

    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])

    index = pd.date_range(startdate, enddate, freq='D')
    timeseriesdf = pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])

    for entry in datalist:
        date = parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            if pd.isna(timeseriesdf.loc[date, column]):
                value = float(entry[column]) if entry[column] is not None else 0.0
                timeseriesdf.loc[date, column] = value

    timeseriesdf.fillna(0.0, inplace=True)
    return timeseriesdf

# Wrangles the England jsondata then assign to another variable
timeseries_df = wrangle_data(jsondata)

In [41]:
# Function to access the API to request the most current England data from the server
def access_api():
    filters = [
        'areaType=nation', 
        'areaName=England'
    ]
    structure = {
        "date": "date",
        "cases": "newCasesBySpecimenDateRollingRate",
        "hospital": "newAdmissionsRollingRate",
        "deaths": "newDailyNsoDeathsByDeathDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    timeseries = api.get_json()
    return timeseries


In [42]:
# Button callback function, retrieves current data when button pressed, uses exceptional handling
def api_button_callback(button):
    while True:
        try:
            apidata = access_api()
            global timeseries_df 
            timeseries_df = wrangle_data(apidata)
            refresh_graph(series.options, scale.options) #fetches the value from widgets below
            apibutton.icon = "check"
            apibutton.disabled = True
            break
        except:
            print('The API request can not be processed at this moment. Please wait temporily before trying again.')
            



## This graph shows Covid Cases, hospital and deaths for England and Scotland. Please press 'Refresh Data' to get current data.

## England

In [43]:
#This wigdets allows the user to control the graph with the three options
series = wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

#Widgets gives user two options for how the graph is displayed
scale = wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

# Function to refresh/display the graph depending what the user clicks
def refresh_graph(gcols, gscale):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        timeseries_df[list(gcols)].plot(logy=logscale)
        plt.show()  
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


controls = wdg.HBox([series, scale]) 

   
graph = wdg.interactive_output(refresh_graph, {'gcols': series, 'gscale': scale})


display(controls, graph) #displays the graph and controls

# Button widget prompting user to refresh data
apibutton = wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='',
    tooltip="Click to download current Public Health England data",
    icon='exclamation-triangle'
)

#When clicked it goes to api_button_callback function
apibutton.on_click(api_button_callback)

# Displays the button
display(apibutton)

Output()

Button(description='Refresh Data', icon='exclamation-triangle', style=ButtonStyle(), tooltip='Click to downloa…

In [44]:
#The Scotland graph is very similar to the previous one, alot of functions can be reused.

with open("scotland.json", "rt") as INFILE:
    jsondata1 = json.load(INFILE)



In [45]:
timeseries_df_scotland = wrangle_data(jsondata1) #can just pass through previous function

In [46]:
def access_api_scotland(): #different function get get data on Scotland
    filters = [
        'areaType=nation',  
        'areaName=Scotland'
    ]
    structure = {
        "date": "date",
        "cases": "newCasesBySpecimenDateRollingRate",
        "hospital": "newAdmissionsRollingRate",
        "deaths": "newDailyNsoDeathsByDeathDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    timeseries_scotland = api.get_json()
    return timeseries_scotland

In [47]:
def api_button_callback_scotland(button): #different function to allow when button clicked to get the data and refresh graph
    while True:
        try:
            apidata_scotland = access_api_scotland()
            timeseries_df_scotland = wrangle_data(apidata_scotland)
            refresh_graph(series_scotland.options, scale_scotland.options)
            apibutton_scotland.icon = "check"
            apibutton_scotland.disabled = True
            break
        except:
            print('The API request can not be processed at this moment. Please wait temporily before trying again.')
            
    
  
        

## Scotland

In [50]:
#very similar to england variables, just names of variables changed to avoid confusion
series_scotland = wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale_scotland = wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

def refresh_graph_scotland(gcols, gscale):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        timeseries_df_scotland[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print("Click to select data for Scotland graph")
        print("(CTRL-Click to select more than one category)")


controls_scotland = wdg.HBox([series_scotland, scale_scotland])

    
graph_scotland = wdg.interactive_output(refresh_graph_scotland, {'gcols': series_scotland, 'gscale': scale_scotland})


display(controls_scotland, graph_scotland)


apibutton_scotland = wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='',
    tooltip="Click to download current Scotland data",
    icon='exclamation-triangle'
)


apibutton_scotland.on_click(api_button_callback_scotland)


display(apibutton_scotland)

Output()

Button(description='Refresh Data', icon='exclamation-triangle', style=ButtonStyle(), tooltip='Click to downloa…